Visual Geometry Group (VGG) Model for Brain MRI Classification

In [ ]:
#Necessary imports
import os
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

c:\Users\rithv\GitHub\brain-mri-scans\code\venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Load and Preprocess Image Data

<div style="font-size:85%">
  
1. Set the folder path for images.  
2. Read the labels CSV file.  
3. Initialize lists for images and labels.  
4. Loop over each row in the DataFrame:  
   1. Get filename and label.  
   2. Construct image path.  
   3. Load image in grayscale.  
   4. Check if image loaded successfully.  
   5. Resize to 64x64.  
   6. Normalize pixel values to [0,1].  
   7. Append to lists.  
5. Convert lists to NumPy arrays.  

</div>

In [2]:
folder_path = "C:/Users/rithv/GitHub/brain-mri-scans/code/images/"

# Load CSV correctly
df = pd.read_csv("labels.csv")   # header row included

images = []
labels = []

for index, row in df.iterrows():
    filename = str(row["filename"])
    label = float(row["target"])

    img_path = os.path.join(folder_path, filename)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print("FAILED TO LOAD:", img_path)
        continue

    img = cv2.resize(img, (64, 64))
    img = img / 255.0

    images.append(img)
    labels.append(label)

x = np.array(images)
y = np.array(labels)



Perform Train Test Split

<div style="font-size:85%">
  
1. Split x and y into training and testing sets with 80/20 ratio.  
2. Use random_state=42 for reproducibility.  

</div>


In [9]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

Define 9-Block VGG Model Training Function

<div style="font-size:85%">
  
1. Define a function to create, compile, train, and evaluate a 9-block VGG model.  
2. Build the Sequential model with 9 convolutional blocks.  
3. Compile with Adam optimizer and binary_crossentropy loss.  
4. Train the model with specified epochs and batch size.  
5. Evaluate on test data and return model, history, and test accuracy.  

</div>

In [1]:
def create_vgg_model(x_train, y_train, x_test, y_test, epochs=10, batch_size=16):
    model = Sequential([
        # Block 1
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(64, 64, 1)),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),

        # Block 2
        Conv2D(64, (3,3), activation='relu', padding='same'),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),

        # Block 3
        Conv2D(128, (3,3), activation='relu', padding='same'),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),

        # Block 4
        Conv2D(256, (3,3), activation='relu', padding='same'),
        Conv2D(256, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),

        # Block 5
        Conv2D(256, (3,3), activation='relu', padding='same'),
        Conv2D(256, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),

        # Block 6
        Conv2D(512, (3,3), activation='relu', padding='same'),
        Conv2D(512, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),

        # Block 7
        Conv2D(512, (3,3), activation='relu', padding='same'),
        Conv2D(512, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),

        # Block 8
        Conv2D(512, (3,3), activation='relu', padding='same'),
        Conv2D(512, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),

        # Block 9
        Conv2D(512, (3,3), activation='relu', padding='same'),
        Conv2D(512, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),

        # Classifier
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        x_train, y_train,
        epochs=epochs, 
        batch_size=batch_size, 
        validation_split=0.2,
        verbose=1
    )

    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    return model, history, accuracy

Train the VGG Model and Print Test Accuracy

In [ ]:
model, history, test_accuracy = create_vgg_model(x_train, y_train, x_test, y_test)

print("Test Accuracy:", test_accuracy)